In [1]:
import numpy as np
from math import sqrt, exp

In [2]:
distanceToStandardDeviation = 0.5

def smooth1D(input, inputSize, distance):
    standardDeviation = distance * distanceToStandardDeviation
    denom = 1 / (2 * standardDeviation * standardDeviation)
    
    out = 0
    weightSum = 0
    for x in range(-inputSize, +inputSize+1):
        d = x*x
        weight = exp(-denom*d)
        out += weight * input[x]
        weightSum += weight
        #print(" x=%d, weight=%f, input=%f -> out=%f"%(x, weight, input[x], weight * input[x]))
    result = out / weightSum
    #print("out=%f, weightSum=%f -> result=%f"%(out, weightSum, result))
    return result

In [3]:
input = np.array([1, 2, 3, 0, 0, -2, -3], dtype=np.float64)
inputSize = 2
distance = 1.5

In [4]:
smoothed = smooth1D(input, inputSize, distance)

In [5]:
def adjSmooth1D(input, inputSize, distance, adjResult):
    # forward
    standardDeviation = distance * distanceToStandardDeviation
    denom = 1 / (2 * standardDeviation * standardDeviation)
    
    out = 0
    weightSum = 0
    for x in range(-inputSize, +inputSize+1):
        d = x*x
        weight = exp(-denom*d)
        out += weight * input[x]
        weightSum += weight
    result = out / weightSum # not needed
        
    # backward
    adjOut = adjResult / weightSum
    adjWeightSum = -adjResult * out / weightSum / weightSum
    print("adjOut=%f, adjWeightSum=%f" %(adjOut, adjWeightSum))
    
    adjDenom = 0
    adjInput = np.zeros(input.shape)
    for x in range(-inputSize, +inputSize+1):
        # forward
        d = x*x
        weight = exp(-denom*d)
        # backward
        adjWeight = adjWeightSum
        adjWeight += adjOut * input[x]
        adjInput[x] += adjOut * weight
        adjDenom -= adjWeight * d * exp(-denom*d)
        print(" x=%d, adjWeight=%f, adjInput=%f, adjDenom=%f"%(x, adjWeight, adjInput[x], -adjWeight * d * exp(-denom*d)))
    
    adjStandardDeviation = -adjDenom / (standardDeviation*standardDeviation*standardDeviation)
    adjDistance = adjStandardDeviation * distanceToStandardDeviation
    print("adjDistance=%f"%adjDistance)
    
    return adjInput, adjDistance

In [6]:
adjResult = 1
adjInput, adjDistance = adjSmooth1D(input, inputSize, distance, adjResult)

adjOut=0.532097, adjWeightSum=-0.174818
 x=-2, adjWeight=-1.239013, adjInput=0.015200, adjDenom=0.141572
 x=-1, adjWeight=-1.771110, adjInput=0.218752, adjDenom=0.728125
 x=0, adjWeight=0.357279, adjInput=0.532097, adjDenom=-0.000000
 x=1, adjWeight=0.889377, adjInput=0.218752, adjDenom=-0.365634
 x=2, adjWeight=1.421474, adjInput=0.015200, adjDenom=-0.162420
adjDistance=-0.404910


In [7]:
def validate(input, inputSize, distance, adjResult, epsilon=1e-7):
    print("Input:", input, "(size=%d)"%inputSize)
    print("Distance:", distance)
    print("adjResult:", adjResult)
    
    # analytic gradient
    anaGradInput, anaGradDistance = adjSmooth1D(input, inputSize, distance, adjResult)
    
    # numeric
    print("Epsilon:", epsilon)
    result = smooth1D(input, inputSize, distance)
    for x in range(-inputSize, +inputSize+1):
        input2 = input.copy()
        #print("input2:", input2)
        input2[x] = input2[x]+epsilon
        #print("input2:", input2)
        res = smooth1D(input2, inputSize, distance)
        numGradInput = (res - result) / epsilon * adjResult
        print("Input[%d], analytic=%f, numeric=%f"%(x, anaGradInput[x], numGradInput))
    distance2 = distance + epsilon
    resD = smooth1D(input, inputSize, distance2)
    numGradDistance = (resD - result) / epsilon * adjResult
    print("Distance, analytic=%f, numeric=%f"%(anaGradDistance, numGradDistance))

In [8]:
validate(input, inputSize, distance, -3)

Input: [ 1.  2.  3.  0.  0. -2. -3.] (size=2)
Distance: 1.5
adjResult: -3
adjOut=-1.596292, adjWeightSum=0.524454
 x=-2, adjWeight=3.717038, adjInput=-0.045599, adjDenom=-0.424716
 x=-1, adjWeight=5.313330, adjInput=-0.656255, adjDenom=-2.184375
 x=0, adjWeight=-1.071838, adjInput=-1.596292, adjDenom=0.000000
 x=1, adjWeight=-2.668130, adjInput=-0.656255, adjDenom=1.096901
 x=2, adjWeight=-4.264422, adjInput=-0.045599, adjDenom=0.487261
adjDistance=1.214731
Epsilon: 1e-07
Input[-2], analytic=-0.045599, numeric=-0.045599
Input[-1], analytic=-0.656255, numeric=-0.656255
Input[0], analytic=-1.596292, numeric=-1.596292
Input[1], analytic=-0.656255, numeric=-0.656255
Input[2], analytic=-0.045599, numeric=-0.045599
Distance, analytic=1.214731, numeric=1.214730


In [9]:
input[-1]

-3.0